In [1]:
import json
import numpy as np
import re
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
#add code to download SQuAD

In [3]:
with open('SQuAD/train-v2.0.json') as file:
    train=json.load(file)
# each paragraph is a different topic indicated by the title
# each topic contains a number of different short passages related to the topic, the context
# for each topic there are ~10 question/answer pairs, and an indicator if a question is impossible

In [4]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split(r'(\W+)?', sent) if x.strip()]

In [5]:
train_context=[]
train_question=[]
train_answer=[]
train_new={'context':train_context,'question':train_question,'answer':train_answer}
for j,data in enumerate(train['data']):
    for i,paragraph in enumerate(data['paragraphs']):
        context=tokenize(paragraph['context'])
        for qas in paragraph['qas']:
            #create a dataset with only the answerable questions
            if (qas['is_impossible']==False):
                train_new['context'].append(context)
                train_new['question'].append(tokenize(qas['question']))
                train_new['answer'].append(tokenize(qas['answers'][0]['text']))

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [11]:
vocab = set()
for context,question,answer in zip(train_new['context'],train_new['question'],train_new['answer']):
#     print(type(context))
#     print(type(question))
#     print(type(answer))
#     vocab |= set(context+question+[str(answer)])
    vocab |= set(context+question+answer)
vocab = sorted(vocab)
vocab_size = len(vocab) + 1
print(np.shape(vocab))

(96585,)


In [7]:
# word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
#needed for sequence padding
story_maxlen = max(map(len, (x for x, _, _ in zip(train_new['context'],train_new['question'],train_new['answer']))))
query_maxlen = max(map(len, (x for _, x, _ in zip(train_new['context'],train_new['question'],train_new['answer']))))

In [8]:
# print(len(word_idx))
print(story_maxlen)
print(query_maxlen)

844
60


In [9]:
def vectorize_stories(data, word_idx, context_maxlen, question_maxlen):
    xs = []
    xqs = []
    ys = []
    for context, question, answer in data:
        x = [word_idx[w] for w in context]
        xq = [word_idx[w] for w in question]
        # let's not forget that index 0 is reserved
        y = 4#np.zeros(len(word_idx) + 1)
#         for string in answer:
#             y[word_idx[string]] = 1
#         print(np.shape(y))
#         print(y)
#         break
        xs.append(x)
        xqs.append(xq)
        ys.append(y)
    return [pad_sequences(xs, maxlen=context_maxlen),
            pad_sequences(xqs, maxlen=question_maxlen), np.array(ys)]